In [1]:
import sklearn
import nltk

from generateWordFrequency import *
from naiveBayes import *
from fileWriteFunctions import *

In [2]:
filename = 'Stopwords.txt'
with open (filename, "r") as myfile:
    data = myfile.readlines()
Stopwords = [word[0:-1] for word in data if len(word[0:-1])]

In [3]:
df_2018 = getDataframe(2018)
df_2018 = addTokenizedColumnofTitle(df_2018,Stopwords)
AllClasses = list(np.unique(df_2018['Post Type']))
AllClasses = ['story', 'ask_hn', 'show_hn', 'poll']
delta = 0.5
appendClassPrefix = 'prob_'
priorProbabilities = getPriorProbabilities(df_2018)
words_2018 = getWordFrequencyDataframe(df_2018,AllClasses)
totalNumberOfWords = getTotalWordCount(words_2018)
words_2018 = obtainDataframeWithClassProbabilities(words_2018, AllClasses, delta, appendClassPrefix)
writeModel(words_2018,'stopword-model.txt',AllClasses,appendClassPrefix)
words_2018 = renameModelRows(words_2018, AllClasses, appendClassPrefix)
model = [words_2018, priorProbabilities]

In [4]:
testData = getDataframe(2019)
testData = addTokenizedColumnofTitle(testData,Stopwords)
testData = testData.reset_index()
testData = generateCondClassProb(testData, model)
testData['predicted'] = generatePrediction(testData,AllClasses)
testResults = comparePredictions(testData,AllClasses)
writeDataframe(testResults,'stopword-result.txt')
check = testResults.comparision.value_counts()
accuracy = check[True]/(check[True]+check[False])
print(accuracy)
df_confusion = pd.crosstab(testResults['Post Type'], testResults['predicted'])
df_confusion['poll'] = 0
print(df_confusion)
df_confusion['precision'] = 0.0
df_confusion['recall'] = 0.0
#df['F1_score'] = 0.0
transpose_matrix = df_confusion.transpose()
for i in transpose_matrix.columns:
    if np.sum(df_confusion[i])==0:
        total = 1
    else:
        total = np.sum(df_confusion[i])
    df_confusion['precision'][i] = df_confusion[i][i]/total
    df_confusion['recall'][i] = df_confusion[i][i]/np.sum(transpose_matrix[i])
df_confusion['F1_score'] = 2/(1/df_confusion['precision']+1/df_confusion['recall'])
print(df_confusion)

0.9898334730167985
predicted  ask_hn  show_hn   story  poll
Post Type                               
ask_hn       5421        4      29     0
poll            1        0       5     0
show_hn        59     4719     125     0
story         421      751  125680     0
predicted  ask_hn  show_hn   story  poll  precision    recall  F1_score
Post Type                                                              
ask_hn       5421        4      29     0   0.918502  0.993949  0.954738
poll            1        0       5     0   0.000000  0.000000  0.000000
show_hn        59     4719     125     0   0.862075  0.962472  0.909511
story         421      751  125680     0   0.998736  0.990761  0.994733


/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/paras/anaconda3/envs/cu/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
